In [1]:
import re
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
path_archivo = "../input_data/KG.csv"
datos_full = pd.read_csv(path_archivo, delimiter = ",")
target = "K"

In [3]:
def scale_column(key, dataSet, par = [], mode = "min-max"):
    if mode == "min-max":
        the_min = min(dataSet[key]) if len(par) == 0 else par[key]["min"]
        the_max = max(dataSet[key]) if len(par) == 0 else par[key]["max"]
        dataSet[key] = (dataSet[key] - the_min)/(the_max - the_min)
    #fin if 
#fin funcion

def determine_columns(datos):
    eig_pattern = r"^(w\d+|eig\d+|eig_\d+|\(omega\^2\)_\d+)$"
    dim_pattern = r"^(dx|dy|dz|bx|by|bz|lx|ly|lz)$"
    rho_pattern = r"^(rho|Rho|RHO)$"
    eig_columns = list(filter(lambda x: re.match(eig_pattern, x), datos.keys()))
    dim_columns = list(filter(lambda x: re.match(dim_pattern, x), datos.keys()))
    rho_columns = list(filter(lambda x: re.match(rho_pattern, x), datos.keys()))
    cols_transformar = eig_columns + dim_columns + rho_columns
    return cols_transformar
#fin función

def get_metrics(X, y, model):
    y_gorro = model.predict(X)
    R2 = r2_score(y, y_gorro)
    RMSE = root_mean_squared_error(y, y_gorro)
    MAE = mean_absolute_error(y, y_gorro)
    return {"R2": R2, "RMSE": RMSE, "MAE": MAE}
#fin función

def get_params(X, mode = "min-max"):
    resp = dict()
    if mode == "min-max":
        for key in X.keys():
            resp[key] = {"min": min(X[key]), "max": max(X[key])}
        #fin for 
    #fin if 
    return resp
#fin función 

 
class TransformadorDeDatos(BaseEstimator, TransformerMixin):
    def __init__(self, dict_op, params = []):
        condicion = callable(dict_op) or all((callable(self.operations[key]) for key in self.operations.keys()))
        if condicion:
            self.operations = dict_op
        else:
            raise TypeError("Class input must be either a callable of a dict full of callables")
        #fin if 
        self.par = params
    #fin init

    def fit(self, X, y=None):
        return self
    #fin fit

    def transform(self, X, post_scaler = "min-max"):
        datos = X.copy()
        cols_transformar = determine_columns(datos)
        if callable(self.operations):
            for col in cols_transformar:
                if col in datos.keys():
                    datos[col] = self.operations(datos[col])
                    scale_column(col, datos, mode = post_scaler, par = self.par)
                #fin if 
            #fin for
        elif type(self.operations) == dict and all((callable(self.operations[key]) for key in self.operations.keys())): 
            for col in cols_transformar:
                if col in datos.keys() and col in self.operations:
                    datos[col] = self.operations[col](datos[col])
                    scale_column(col, datos, mode = post_scaler, par = self.par)
                #fin if 
            #fin for
        else: 
            raise TypeError("Class input must be either a callable of a dict full of callables")
        #fin if 
        return datos
    #fin transform
#fin class

In [4]:
orden = [("custom_transformer", TransformadorDeDatos(dict_op=np.log)),
         ("lin-reg", LinearRegression())]
pipeline = Pipeline(orden)

In [5]:
features = determine_columns(datos_full)
X_full = datos_full[features]
y_full = datos_full[target]
pipeline.fit(X_full, y_full)
y_gorro_full = pipeline.predict(X_full)

In [6]:
primer_transformer = pipeline.named_steps["custom_transformer"]
X_tranf = primer_transformer.transform(X_full)
X_tranf.head()

,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,dx,dy,dz,rho
0,0.817614,0.830022,0.827608,0.830971,0.830971,0.820179,0.820039,0.822732,0.823866,0.822408,0.0,0.0,0.0,0.0
1,0.871076,0.866716,0.871217,0.874975,0.881228,0.874864,0.875924,0.870402,0.870599,0.869217,0.0,0.0,0.0,0.0
2,0.878663,0.874560,0.877614,0.886671,0.897695,0.885200,0.893723,0.888088,0.895079,0.893737,0.0,0.0,0.0,0.0
3,0.882043,0.878055,0.878616,0.889105,0.907947,0.899394,0.902559,0.896869,0.904482,0.903155,0.0,0.0,0.0,0.0
4,0.884006,0.880084,0.879213,0.890491,0.913183,0.908723,0.908652,0.906841,0.910947,0.909631,0.0,0.0,0.0,0.0


In [7]:
X_full.head()

,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,dx,dy,dz,rho
0,6.846532,8.279029,8.279029,8.607635,8.607635,9.277155,9.277155,10.018620,10.188246,10.188246,0.1,0.1,0.1,0.2
1,12.049711,12.049711,12.852182,13.398173,14.267764,15.790196,15.968844,15.968844,16.066732,16.066732,0.1,0.1,0.1,0.2
2,13.056281,13.056281,13.708609,15.070374,16.836928,17.460062,18.984130,18.984130,20.396466,20.396466,0.1,0.1,0.1,0.2
3,13.531380,13.531380,13.847906,15.443661,18.665271,20.044593,20.686383,20.686383,22.354199,22.354199,0.1,0.1,0.1,0.2
4,13.815095,13.815095,13.931538,15.660428,19.674186,21.948089,21.948089,22.805466,23.808203,23.808203,0.1,0.1,0.1,0.2


In [8]:
print((np.log(X_full["w0"][0])-min(np.log(X_full["w0"])))/(max(np.log(X_full["w0"])- min(np.log(X_full["w0"])))))

0.8176139416371806


In [9]:
metricas_full = get_metrics(X_full, y_full, pipeline)

In [10]:
print(metricas_full)

{'R2': 0.08550190502541999, 'RMSE': 1.659009541687749, 'MAE': 1.428676898610909}


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size = 0.33)

In [12]:
param_train = get_params(X_train)
orden2 = [("custom_transformer", TransformadorDeDatos(dict_op=np.log, params=param_train)),
         ("lin-reg", LinearRegression())]
pipeline2 = Pipeline(orden2)
pipeline2.fit(X_train, y_train)
metricas2 = get_metrics(X_test, y_test, pipeline2)

In [13]:
print(metricas2)

{'R2': 0.08284314600337461, 'RMSE': 1.6618525017938477, 'MAE': 1.429834502775277}
